In [1]:
import numpy as np
from itertools import chain, combinations

# Computing Geometric Products of Multivectors in $\mathbb{G}^n$

Notation note: we use ``vector`` to refer to things in $\mathbb{R}^n$, ``scalar`` to refer to things in $\mathbb{R}$, and ``multivector`` to refer to things in $\mathbb{G}^n$. So $e_1$ is a vector, $e_1e_2$ is a multivector.

We want to implement the geometric product of two multivectors in $\mathbb{G}^n$. For all our concrete examples, we will use $n = 3$, but our implementation can handle general $n$. Remember that computing the geometric product of multivectors written out as a linear combination of orthonormal basis elements is done by FOILing, then using the facts
1. scalars commute freely;
2. the geometric product is associative;
3. the geometric product of orthogonal vectors is anticommutative;
4. $e_ke_k = 1$.

For example, let 
\begin{align*}
  M_1 &= 2 + e_1 + 3e_3 - 2e_1e_2\\
  M_2 &= 2e_2e_3 - e_1e_2e_3.
\end{align*}

Then we compute the geometric product as follows:
\begin{align*}
  M_1M_2 &= (2 + e_1 + 3e_3 - 2e_1e_2)(2e_2e_3 - e_1e_2e_3)\\
  &= 4e_2e_3 - 2e_1e_2e_3 + 2e_1e_2e_3 - e_1e_1e_2e_3 + 6e_3e_2e_3 - 3e_3e_1e_2e_3 - e_1e_2e_2e_3 + 2e_1e_2e_1e_2e_3\\
  &= 4e_2e_3 - e_2e_3 - 6e_2 - 3e_1e_2 - 4e_1e_2 - 2e_3\\
  &= -6e_2 - 2e_3 - 3e_1e_2 - 4e_1e_3 + 3e_2e_3
\end{align*}
Note that the basis vectors in $\mathbb{G}^n$ are given by the power set of $\{e_1, \dots, e_n\}$. The function `basis(n)` computes a list of all basis vectors, represented as tuples, e.g., $e_1e_3$ is represented as `(1, 3)`. The basis vector $1$ is represented by the empty tuple `()`. We adopt a convention of ordering the basis vectors witha dictionary ordering. Therefore the product $M_1M_2$ can be represented in coordinates as `[0, 0, -6, -2, -3, -4, 3, 0]`

In [2]:
def basis(n):
  # compute a basis for G^n with tuple representation
  # stolen from https://stackoverflow.com/questions/1482308/how-to-get-all-subsets-of-a-set-powerset
  s = list(range(1, n + 1))
  return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

In [3]:
basis(3)

[(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

We want to be able to convert between the tuple representation of a basis vector and its one-hot coordinate representation, and vice-versa. We call tuples ``tooples`` in code because ``tuple`` is a reserved word in Python.

In [4]:
def toople_to_coords(toople, n):
  '''
  (1, 2), n = 2 -> [0, 0, 0, 1]
  (1, 2), n = 3 -> [0, 0, 0, 0, 1, 0, 0, 0]
  toople must be a tuple representing a basis element in G^n
  '''
  assert len(toople) <= n
  coords = np.zeros(shape = 2**n) # coordinate representation
  hot_idx = basis(n).index(toople) # find the index of the tuple in the list of basis vectors
  coords[hot_idx] = 1
  return coords
  
def coords_to_toople(coords):
  # check that arr is one-hot
  coords = np.array(coords)
  n = int(np.log2(coords.size))
  assert np.sum(coords) == 1
  assert np.count_nonzero(coords) == 1

  hot_idx = np.nonzero(coords)[0][0] # get the idx of the one
  toople = basis(n)[hot_idx]
  return toople

The coordinate representation of $e_1e_3$ in $\mathbb{G}^3$ should be `[0, 0, 0, 0, 0, 1, 0, 0]`. The coordinate representation of $1$ in $\mathbb{G}^3$ should be `[1, 0, 0, 0, 0, 0, 0, 0]`.

In [5]:
print(f"e_1e_3 as coordinates: {toople_to_coords((1, 3), 3)}")
print(f"e_1e_3 as a tuple: {coords_to_toople(np.array([0, 0, 0, 0, 0, 1, 0, 0]))}")
print(f"e_1e_3 as coordinates: {toople_to_coords((), 3)}")
print(f"e_1e_3 as a tuple: {coords_to_toople(np.array([1, 0, 0, 0, 0, 0, 0, 0]))}")

e_1e_3 as coordinates: [0. 0. 0. 0. 0. 1. 0. 0.]
e_1e_3 as a tuple: (1, 3)
e_1e_3 as coordinates: [1. 0. 0. 0. 0. 0. 0. 0.]
e_1e_3 as a tuple: ()


Now we want to be able to simplify the geometric product of basis elements, i.e., use properties 2, 3, and 4 above to reduce a string of basis vectors. We stick to the convention that we always write a product of basis elements in order, e.g. we would always write $-e_1e_2$ instead of $e_2e_1$. If we only consider multiplying basis vectors, that means the concatenation of two strings of basis multivectors would at most contain two instances of the same vector. That means that up to sign, the product of two basis multivectors consists of exactly the basis vectors that are in the symmetric difference of the two multivectors. This is because if a vector were in both basis multivectors, it would occur exactly once in each basis multivector, and swapping elements would eventually kill that vector; if a vector were in neither basis multivector, it wouldn't occur in the product; and if a vector were in exactly one of the basis multivectors, it would not be killed.

To get the sign of a multivector, notice that the most human way (to me, at least) to compute the product of two basis multivectors is to do a bubble sort, swapping the sign of the product every time we swap adjacent vectors. So we can compute the sign of a product of two multivectors by bubble sorting the concatenatation of the strings, counting how many swaps we do. This is probably a stupid method but for small $n$ whatever. 

We decouple the sign from the string of vectors, since we represent the 1 basis vector as `()` and we can't have a negative empty tuple.

In [6]:
def count_swaps(arr):
  # count the number of swaps needed to bubble sort an array
  arr = np.array(arr)
  n = arr.size
  num_swaps = 0
  while not np.all(arr[:-1] <= arr[1:]): # while the array is not sorted
    for i in range(n - 1):
      for j in range(n - i - 1):
        if arr[j] > arr[j + 1]:
          arr[j], arr[j + 1] = arr[j + 1], arr[j]
          num_swaps += 1
  return num_swaps

def compute_basis_gp(basis1, basis2):
  # basis1, basis2 are just arrays of indices. so [1, 3] = e1e3. bases are assumed sorted and contaning no repeating indices.
  # product of basis multivectors up to sign is just the indices in the symmetric difference
  basis1 = np.array(basis1)
  basis2 = np.array(basis2)
  sgn = (-1)**count_swaps(np.concatenate((basis1, basis2)))
  
  prod = np.sort(np.array(list(set(basis1) ^ set(basis2)))) # magic 

  prod = tuple(prod)
  return sgn, prod

As a sanity check, we should have $(e_1e_2)(e_1e_2e_3) = -e_3$:

In [7]:
basis1 = [1, 2]
basis2 = [1, 2 ,3]

compute_basis_gp(basis1, basis2)

(-1, (3,))

Now we want to compute a lookup table of all products between the $2^n$ basis multivectors in $\mathbb{G}^3$. This will be $2^n \times 2^n\times n$ shaped array, where the $i, j$ entry is the $n$-dimensional one-hot representation of the product of the $i$th and $j$th basis multivector (remember we are using a flattened dictionary ordering).

In [8]:
def compute_lookup_table(n):
  # compute the 2^n x 2^n x n lookup table
  lookup_table = np.zeros(shape = (2**n, 2**n, 2**n,))
  tooples = basis(n)
  for i in range(2**n):
    for j in range(2**n):
      sgn, prod = compute_basis_gp(tooples[i], tooples[j]) # prod is a tuple, sgn is +-1
      lookup_table[i, j] = sgn * toople_to_coords(prod, n)
  return lookup_table

You can check for yourself that the lookup table for $\mathbb{G}^3$ is as follows.

|                    | $\bold{1}$  | $\bold{e_1}$ | $\bold{e_2}$ | $\bold{e_3}$ | $\bold{e_1e_2}$ | $\bold{e_1e_3}$ | $\bold{e_2e_3}$ | $\bold{e_1e_2e_3}$ |
|--------------------|-------------|--------------|--------------|--------------|-----------------|-----------------|-----------------|--------------------|
| $\bold{1}$         | $1$         | $e_1$        | $e_2$        | $e_3$        | $e_1e_2$        | $e_1e_3$        | $e_2e_3$        | $e_1e_2e_3$        |
| $\bold{e_1}$       | $e_1$       | $1$          | $e_1e_2$     | $e_1e_3$     | $e_2$           | $e_3$           | $e_1e_2e_3$     | $e_2e_3$           |
| $\bold{e_2}$       | $e_2$       | $-e_1e_2$    | $1$          | $e_2e_3$     | $-e_1$          | $-e_1e_2e_3$    | $e_3$           | $-e_1e_3$          |
| $\bold{e_3}$       | $e_3$       | $-e_1e_3$    | $-e_2e_3$    | $1$          | $e_1e_2e_3$     | $-e_1$          | $-e_2$          | $e_1e_2$           |
| $\bold{e_1e_2}$    | $e_1e_2$    | $-e_2$       | $e_1$        | $e_1e_2e_3$  | $-1$            | $-e_2e_3$       | $e_1e_3$        | $-e_3$             |
| $\bold{e_1e_3}$    | $e_1e_3$    | $-e_3$       | $-e_1e_2e_3$ | $e_1$        | $e_2e_3$        | $-1$            | $-e_1e_2$       | $e_2$              |
| $\bold{e_2e_3}$    | $e_2e_3$    | $e_1e_2e_3$  | $-e_3$       | $e_2$        | $-e_1e_3$       | $e_1e_2$        | $-1$            | $-e_1$             |
| $\bold{e_1e_2e_3}$ | $e_1e_2e_3$ | $e_2e_3$     | $-e_1e_3$    | $e_1e_2$     | $-e_3$          | $e_2$           | $-e_1$          | $-1$               |

Let's print out our computed lookup table as tuples. You can verify that it matches.

In [9]:
lookup_table_str = []

lookup_table = compute_lookup_table(3)

for i in range(8):
  row = []
  for j in range(8):
    if np.all(lookup_table[i, j] >= 0):
      sgn = " "
    else:
      sgn = "-"
    row.append(sgn + str(coords_to_toople(np.abs(lookup_table[i, j]))))
  lookup_table_str.append(row)

# pretty print 2D list from https://stackoverflow.com/questions/13214809/pretty-print-2d-list
s = [[str(e) for e in row] for row in lookup_table_str]
lens = [max(map(len, col)) for col in zip(*s)]
fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
table = [fmt.format(*row) for row in s]
print('\n'.join(table))

 ()       	 (1,)     	 (2,)     	 (3,)     	 (1, 2)   	 (1, 3)   	 (2, 3)   	 (1, 2, 3)
 (1,)     	 ()       	 (1, 2)   	 (1, 3)   	 (2,)     	 (3,)     	 (1, 2, 3)	 (2, 3)   
 (2,)     	-(1, 2)   	 ()       	 (2, 3)   	-(1,)     	-(1, 2, 3)	 (3,)     	-(1, 3)   
 (3,)     	-(1, 3)   	-(2, 3)   	 ()       	 (1, 2, 3)	-(1,)     	-(2,)     	 (1, 2)   
 (1, 2)   	-(2,)     	 (1,)     	 (1, 2, 3)	-()       	-(2, 3)   	 (1, 3)   	-(3,)     
 (1, 3)   	-(3,)     	-(1, 2, 3)	 (1,)     	 (2, 3)   	-()       	-(1, 2)   	 (2,)     
 (2, 3)   	 (1, 2, 3)	-(3,)     	 (2,)     	-(1, 3)   	 (1, 2)   	-()       	-(1,)     
 (1, 2, 3)	 (2, 3)   	-(1, 3)   	 (1, 2)   	-(3,)     	 (2,)     	-(1,)     	-()       


Finally, we want a clever numpythonic way of computing arbitrary geometric products. Let's FOIL out the geometric product of two generic multivectors in $\mathbb{G}^2$ ($\mathbb{G}^3$ would be a nightmare) in a suggestive manner:
\begin{align*}
  (a_0 &+ a_1e_1 + a_2e_2 + a_3e_1e_2)(b_0 + b_1e_1 + b_2e_2 + b_3e_1e_2) \\
  & = a_0b_0 + a_0b_1e_1 + a_0b_2e_2 + a_0b_3e_1e_2\\
  & + a_1b_0e_1 + a_1b_1e_1 + a_1b_2e_1e_2 + a_1b_3e_1e_1e_2\\
  & = a_2b_0e_2 + a_2b_1e_1e_1 + a_2b_2e_1e_2 + a_2b_3e_2e_1e_2\\
  & = a_3b_0e_1e_2 + a_3b_1e_1e_2e_1  + a_3b_2e_1e_2e_2 + a_3b_3e_1e_2e_1e_2.
\end{align*}
If you stare at this for a bit, you can hopefully convince yourself that this is the Frobenius inner product of the element-wise product of the tensor product of the coordinate vectors with the lookup table. In notation, let $a = (a_0, a_1, a_2, a_3)$ and $b = (b_0, b_1, b_2, b_3)$, and let $A$ and $B$ be the multivectors with coordinates $a$ and $b$. Then 
$$AB = \langle{a\otimes b}, \text{lookup table}\rangle.$$
Note that we are abusing notation since the lookup table is really a 3-dimensional tensor. The implementation of the geometric product is now simple.

In [10]:
def geom_prod(M1, M2, lookup_table):
  # M1, M2 as flattened coordinate representations of multivectors
  kron = np.einsum("i, j -> ij", M1, M2)
  res = np.einsum("ij, ijk -> k", kron, lookup_table)
  return res

Let's confirm our computation done by hand at the beginning:
$$(2 + e_1 + 3e_3 - 2e_1e_2)(2e_2e_3 - e_1e_2e_3) = -6e_2 - 2e_3 - 3e_1e_2 - 4e_1e_3 + 3e_2e_3.$$
Remember the two multivectors on the left have coordinate representations `[2, 1, 0, 3, -2, 0, 0, 0]` and `[0, 0, 0, 0, 0, 0, 2, -1]`, while the multivector on the right has coordinate representation `[0, 0, -6, -2, -3, -4, 3, 0]`.

In [11]:
lookup_table = compute_lookup_table(3)

In [12]:
M1 = np.array([2, 1, 0, 3, -2, 0, 0, 0])
M2 = np.array([0, 0, 0, 0, 0, 0, 2, -1])
geom_prod(M1, M2, lookup_table)

array([ 0.,  0., -6., -2., -3., -4.,  3.,  0.])

Yay!